In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

X = pickle.load(open('X.pickle','rb'))
y = pickle.load(open('y.pickle','rb'))
X_t = pickle.load(open('X_t.pickle','rb'))
y_t = pickle.load(open('y_t.pickle','rb'))

In [2]:
X = X/255.0
X_t = X_t/255.0

In [3]:
model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation('sigmoid'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
model.fit(X, y, batch_size=32, epochs=5, validation_split=0.3)

Epoch 1/5
21/21 [==============================] - 131s 6s/step - loss: 0.7608 - accuracy: 0.5477 - val_loss: 0.6273 - val_accuracy: 0.5922
Epoch 2/5
21/21 [==============================] - 120s 6s/step - loss: 0.6050 - accuracy: 0.6802 - val_loss: 0.4362 - val_accuracy: 0.8511
Epoch 3/5
21/21 [==============================] - 132s 6s/step - loss: 0.3224 - accuracy: 0.8966 - val_loss: 0.1504 - val_accuracy: 0.9574
Epoch 4/5
21/21 [==============================] - 130s 6s/step - loss: 0.1106 - accuracy: 0.9705 - val_loss: 0.0330 - val_accuracy: 0.9965
Epoch 5/5
21/21 [==============================] - 130s 6s/step - loss: 0.0312 - accuracy: 0.9977 - val_loss: 0.0228 - val_accuracy: 1.0000


In [8]:
val_loss, val_acc = model.evaluate(X_t, y_t)
print(val_loss)
print(val_acc)

9/9 [==============================] - 10s 1s/step - loss: 0.0742 - accuracy: 0.9728
0.07424554228782654
0.9727626442909241


In [9]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

Saved model to disk


In [12]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [16]:
# Checking if model is saved correctly

from tensorflow.keras.models import model_from_json

loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
accuracy: 99.36%
